<a href="https://colab.research.google.com/github/Swayamprakashpatel/PU_DL/blob/main/PU_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**# STEP - 1 : LOAD POWDER UNIFORMITY DEEP LEARNING MODEL**

(Download and extract the Authentication File Kaggle.json from [HERE](https://github.com/Swayamprakashpatel/PU_DL.git) )

In [ ]:
#@title Select kaggle.json file when prompt "choose file" ()
import random
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.imagenet_utils import decode_predictions

from google.colab import files

#@title Prediction 
#Prediction
#%%capture

from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

#Make Directory of Kaggle and set its permission for access.
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

# Download Data from Kaggle Fast and Unzip them in /content
!kaggle datasets download -d drswayamprakashpatel/powderblend -p /content

#Unzip data (Two Folders - Training and Validation)
import os
os.chdir('/content')
#create a directory named train/
!unzip -q powderblend.zip


**#STEP -2: LOAD MICROSCOPIC IMAGES FOR PREDICTION**

In [ ]:

#@title Selecct Image/Images (.jpg format only) when promt "choose file" 

!mkdir -p /content/Prediction/Images
files.upload()
!mv *.jpg /content/Prediction/Images


batch_size = 10
IMG_HEIGHT, IMG_WIDTH = (1000,1000)

Pred_Dir = '/content/Prediction'
Prediction = ImageDataGenerator(rescale = 1./255)
Pred_Data =  Prediction.flow_from_directory(
                    directory=Pred_Dir,
                    batch_size= batch_size,
                    shuffle=False,
                    target_size=(IMG_HEIGHT,IMG_WIDTH),
                    class_mode=None)
#Pred_Data.reset()
from keras.models import load_model

model = load_model('/content/DL_Model.hdf5')
Prediction = np.round(model.predict(Pred_Data))
Prediction = np.any(Prediction, axis=1)


#Multiple images' prediction with image caption

labels = ({'MIXED': 0, 'NOT MIXED': 1})
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in Prediction]
filenames = Pred_Data.filenames
results = pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
%rm -rf Prediction
print(results)